In [2]:
from scipy.io import arff
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import math
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from itertools import product
from sklearn.model_selection import cross_val_score
from fastprogress.fastprogress import progress_bar
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from fastprogress.fastprogress import progress_bar
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from sklearn.svm import LinearSVC
from tabulate import tabulate
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE


In [3]:
df = pd.read_csv("Frogs_MFCCs.csv")

train, test = train_test_split(df, test_size=0.3)


Research exact match and hamming score/ loss methods for evaluating multi-
label classication and use them in evaluating the classiers in this problem

Train a SVM for each of the labels, using Gaussian kernels and one versus all
classiers. Determine the weight of the SVM penalty and the width of the
Gaussian Kernel using 10 fold cross validation.1

In [33]:

X_train = train.drop(columns=['Family','Genus','Species','RecordID'])
normalized_x_train_df=(X_train-X_train.mean())/X_train.std()
y_train=train[['Family','Genus','Species']]
X_test = test.drop(columns=['Family','Genus','Species','RecordID'])
normalized_x_test_df=(X_test-X_test.mean())/X_test.std()
y_test=test[['Family','Genus','Species']]


In [34]:
gamma_range=np.array([0.1,0.5,1,1.5,2,2.5, 10])
c_range=np.array([0.1,1, 10, 100])

In [35]:
svc = SVC(kernel='rbf')
clf = GridSearchCV(estimator=svc, param_grid=dict(C=c_range,gamma=gamma_range), n_jobs=-1,cv=10)
clf.fit(normalized_x_train_df, y_train['Family'])
print(clf.best_estimator_.gamma ,clf.best_estimator_.C ,clf.best_score_)
c_family=clf.best_estimator_.C 
g_family=clf.best_estimator_.gamma
family_cv_score=clf.best_score_

0.1 10.0 0.9898762188772128


In [37]:
svc = SVC(kernel='rbf')
clf = GridSearchCV(estimator=svc, param_grid=dict(C=c_range,gamma=gamma_range), n_jobs=-1,cv=10)
clf.fit(normalized_x_train_df, y_train['Genus'])
print(clf.best_estimator_.gamma ,clf.best_estimator_.C ,clf.best_score_)
c_genus=clf.best_estimator_.C 
g_genus=clf.best_estimator_.gamma
genus_cv_score=clf.best_score_

0.1 10.0 0.988087348922339


In [38]:
svc = SVC(kernel='rbf')
clf = GridSearchCV(estimator=svc, param_grid=dict(C=c_range,gamma=gamma_range), n_jobs=-1,cv=10)
clf.fit(normalized_x_train_df, y_train['Species'])
print(clf.best_estimator_.gamma ,clf.best_estimator_.C ,clf.best_score_)
c_species=clf.best_estimator_.C 
g_species=clf.best_estimator_.gamma
species_cv_score=clf.best_score_

0.1 10.0 0.9853060210167568


In [59]:
svc = SVC(kernel='rbf',C=c_family,gamma=g_family)
svc.fit(normalized_x_train_df, y_train['Family'])
y_pred_family=svc.predict(normalized_x_test_df)
family_accuracy_score=accuracy_score(y_test['Family'], y_pred_family)
family_hamming_loss=hamming_loss(y_test['Family'], y_pred_family)

svc = SVC(kernel='rbf',C=c_genus,gamma=g_genus)
svc.fit(normalized_x_train_df, y_train['Genus'])
y_pred_genus=svc.predict(normalized_x_test_df)
genus_accuracy_score=accuracy_score(y_test['Genus'], y_pred_genus)
genus_hamming_loss=hamming_loss(y_test['Genus'], y_pred_genus)

svc = SVC(kernel='rbf',C=c_species,gamma=g_species)
svc.fit(normalized_x_train_df, y_train['Species'])
y_pred_species=svc.predict(normalized_x_test_df)
species_accuracy_score=accuracy_score(y_test['Species'], y_pred_species)
species_hamming_loss=hamming_loss(y_test['Species'], y_pred_species)

In [57]:
table = [["C Value",c_family,c_genus,c_species],["Gamma",g_family,g_genus,g_species],["CV Score",family_cv_score,genus_cv_score,species_cv_score], ["Hamming Loss",family_hamming_loss,genus_hamming_loss,species_hamming_loss],["Test Accuracy SCore",family_accuracy_score,genus_accuracy_score,species_accuracy_score]]
print(tabulate(table,headers=["","Family", "Genus","Species"],tablefmt="presto"))

print('avg accuracy score :',(family_accuracy_score+genus_accuracy_score+species_accuracy_score)/3)
print('avg Hamming Loss :',(family_hamming_loss+genus_hamming_loss+species_hamming_loss)/3)

                     |     Family |      Genus |    Species
---------------------+------------+------------+------------
 C Value             | 10         | 10         | 10
 Gamma               |  0.1       |  0.1       |  0.1
 CV Score            |  0.989876  |  0.988087  |  0.985306
 Hamming Loss        |  0.0847615 |  0.0921723 |  0.0477073
 Test Accuracy SCore |  0.915239  |  0.907828  |  0.952293
avg accuracy score : 0.9251196541608769
avg Hamming Loss : 0.07488034583912305


clf.classes_

Repeat 1(b)ii with L1-penalized SVMs.3 Remember to normalize the at-
tributes.
This time I am trying GridSearchCV

In [41]:
Cs =[1,5,10,100]
svc = LinearSVC(penalty='l1', dual=False)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(normalized_x_train_df, y_train['Family'])
print(clf.best_estimator_.C ,clf.best_score_)
c_l1_fam=clf.best_estimator_.C
cv_l1_fam=clf.best_score_

1 0.936459072208824


In [42]:

svc = LinearSVC(penalty='l1', dual=False)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(normalized_x_train_df, y_train['Genus'])
print(clf.best_estimator_.C ,clf.best_score_)
c_l1_gen=clf.best_estimator_.C
cv_l1_gen=clf.best_score_

5 0.9529403303857127


In [43]:
svc = LinearSVC(penalty='l1', dual=False)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(normalized_x_train_df, y_train['Species'])
print(clf.best_estimator_.C ,clf.best_score_)
c_l1_spc=clf.best_estimator_.C
cv_l1_spc=clf.best_score_

100 0.9608823552592172


In [63]:
svc = LinearSVC(penalty='l1', dual=False, C=c_l1_fam)
svc.fit(normalized_x_train_df, y_train['Family'])
y_pred_family=svc.predict(normalized_x_test_df)
family_accuracy_score=accuracy_score(y_test['Family'], y_pred_family)
family_hamming_loss=hamming_loss(y_test['Family'], y_pred_family)

svc = LinearSVC(penalty='l1', dual=False,C=c_l1_gen)
svc.fit(normalized_x_train_df, y_train['Genus'])
y_pred_genus=svc.predict(normalized_x_test_df)
genus_accuracy_score=accuracy_score(y_test['Genus'], y_pred_genus)
genus_hamming_loss=hamming_loss(y_test['Genus'], y_pred_genus)

svc = LinearSVC(penalty='l1', dual=False,C=c_l1_spc)
svc.fit(normalized_x_train_df, y_train['Species'])
y_pred_species=svc.predict(normalized_x_test_df)
species_accuracy_score=accuracy_score(y_test['Species'], y_pred_species)
species_hamming_loss=hamming_loss(y_test['Species'], y_pred_species)

In [64]:
table = [["C Value",c_l1_fam,c_l1_gen,c_l1_spc],["CV Score",cv_l1_fam,cv_l1_gen,cv_l1_spc], ["Hamming Loss",family_hamming_loss,genus_hamming_loss,species_hamming_loss],["Test Accuracy SCore",family_accuracy_score,genus_accuracy_score,species_accuracy_score]]
print(tabulate(table,headers=["","Family", "Genus","Species"],tablefmt="presto"))
print('avg accuracy score :',(family_accuracy_score+genus_accuracy_score+species_accuracy_score)/3)
print('avg Hamming Loss :',(family_hamming_loss+genus_hamming_loss+species_hamming_loss)/3)

                     |    Family |    Genus |     Species
---------------------+-----------+----------+-------------
 C Value             | 1         | 5        | 100
 CV Score            | 0.936459  | 0.95294  |   0.960882
 Hamming Loss        | 0.0643817 | 0.052339 |   0.0453914
 Test Accuracy SCore | 0.935618  | 0.947661 |   0.954609
avg accuracy score : 0.9459626370233133
avg Hamming Loss : 0.05403736297668674


Repeat 1(b)ii with L1-penalized SVMs.3 Remember to normalize the at-
tributes.

In [47]:
svc = LinearSVC(penalty='l1', dual=False)
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(normalized_x_train_df, y_train['Family'])
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(X, y)
print(clf.best_estimator_.C ,clf.best_score_)
c_smote_fam=clf.best_estimator_.C
cv_smote_fam=clf.best_score_

10 0.9488224677525567


In [54]:
svc = LinearSVC(penalty='l1', dual=False)
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(normalized_x_train_df, y_train['Genus'])
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(X, y)
print(clf.best_estimator_.C ,clf.best_score_)
c_smote_gen=clf.best_estimator_.C
cv_smote_gen=clf.best_score_

100 0.9597051760724661


In [49]:
svc = LinearSVC(penalty='l1', dual=False)
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(normalized_x_train_df, y_train['Species'])
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(X, y)
print(clf.best_estimator_.C ,clf.best_score_)
c_smote_spc=clf.best_estimator_.C
cv_smote_spc=clf.best_score_

5 0.9665444172779136


In [65]:
svc = LinearSVC(penalty='l1', dual=False, C=c_smote_fam)
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(normalized_x_train_df, y_train['Family'])
svc.fit(X,y)
y_pred_family=svc.predict(normalized_x_test_df)
family_accuracy_score=accuracy_score(y_test['Family'], y_pred_family)
family_hamming_loss=hamming_loss(y_test['Family'], y_pred_family)

svc = LinearSVC(penalty='l1', dual=False,C=c_smote_gen)
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(normalized_x_train_df, y_train['Genus'])
svc.fit(X,y)
y_pred_genus=svc.predict(normalized_x_test_df)
genus_accuracy_score=accuracy_score(y_test['Genus'], y_pred_genus)
genus_hamming_loss=hamming_loss(y_test['Genus'], y_pred_genus)

svc = LinearSVC(penalty='l1', dual=False,C=c_smote_spc)
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(normalized_x_train_df, y_train['Species'])
svc.fit(X,y)
y_pred_species=svc.predict(normalized_x_test_df)
species_accuracy_score=accuracy_score(y_test['Species'], y_pred_species)
species_hamming_loss=hamming_loss(y_test['Species'], y_pred_species)


In [66]:
table = [["C Value",c_smote_fam,c_smote_gen,c_smote_spc],["CV Score",cv_smote_fam,cv_smote_gen,cv_smote_spc], ["Hamming Loss",family_hamming_loss,genus_hamming_loss,species_hamming_loss],["Test Accuracy SCore",family_accuracy_score,genus_accuracy_score,species_accuracy_score]]
print(tabulate(table,headers=["","Family", "Genus","Species"],tablefmt="presto"))
print('avg accuracy score :',(family_accuracy_score+genus_accuracy_score+species_accuracy_score)/3)
print('avg Hamming Loss :',(family_hamming_loss+genus_hamming_loss+species_hamming_loss)/3)

                     |     Family |       Genus |   Species
---------------------+------------+-------------+-----------
 C Value             | 10         | 100         | 5
 CV Score            |  0.959705  |   0.959705  | 0.966544
 Hamming Loss        |  0.0847615 |   0.0921723 | 0.0477073
 Test Accuracy SCore |  0.915239  |   0.907828  | 0.952293
avg accuracy score : 0.9251196541608769
avg Hamming Loss : 0.07488034583912305
